In [ ]:
!pip install beir==1.0.0

In [ ]:
from beir import LoggingHandler, util
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.evaluation import EvaluateRetrieval

dataset = "nfcorpus"
data_root_dir = "beir_data"

url = f"https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{dataset}.zip"
data_path = util.download_and_unzip(url, data_root_dir)
corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")

In [ ]:
from opensearchpy import OpenSearch

client = OpenSearch(hosts=["localhost:9200"])
index_name = "test"

In [ ]:
from tqdm import tqdm

i=0
bulk_body=[]
for _id,body in tqdm(corpus.items()):
    text=body["title"]+" "+body["text"]
    bulk_body.append({ "index" : { "_index" : index_name, "_id" : _id } })
    bulk_body.append({ "text" : text })
    i+=1
    if i%50==0:
        response=client.bulk(bulk_body,request_timeout=100)
        assert response["errors"]==False
        try:
            assert response["errors"]==False
        except:
            print("there is errors")
            time.sleep(1)
            response = client.bulk(bulk_body,request_timeout=100)
            assert response["errors"]==False
        bulk_body=[]
        
response=client.bulk(bulk_body,request_timeout=100)
assert response["errors"]==False
client.indices.refresh(index=index_name)

In [ ]:
run_res={}
for _id,text in tqdm(queries.items()):
    query={
      'size': 20,
      'query': {
        "match": {
            "text":text
        }
      }
    }
    response=client.search(index=index_name,body=query)
    hits=response["hits"]["hits"]
    run_res[_id]={item["_id"]:item["_score"] for item in hits}
for query_id, doc_dict in tqdm(run_res.items()):
    if query_id in doc_dict:
        doc_dict.pop(query_id)
res=EvaluateRetrieval.evaluate(qrels, run_res, [4,10])
print(res)